# 1. DATA EXTRACTION CODE

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
from tqdm import tqdm  # Optional: Provides a progress bar
import re

In [2]:
def extract_person_occupations(xml_file):
    occupations = []
    with open(xml_file, 'r', encoding='utf-8') as f:
        content = f.read()
        
        start_tag = '<span role="occupation">'
        end_tag = '</span>'
        start_index = content.find(start_tag)
        
        while start_index != -1:
            start_index += len(start_tag)
            end_index = content.find(end_tag, start_index)
            occupation = content[start_index:end_index].strip()
            occupations.append(occupation)
            start_index = content.find(start_tag, end_index)

    return occupations

def extract_names(xml_file, print_names=False):
    names = {
        'Main Name': 'N/A',
        'Forenames': 'N/A',
        'ShortForm': 'N/A',
        'Alternative Names': []
    }

    def extract_name(name_section, predicate):
        name_elem = name_section.find(f".//metaDescribes[@predicate='{predicate}']")
        return name_elem.text if name_elem is not None else 'N/A'
    tree = ET.parse(xml_file)
    root = tree.getroot()
    name_section = root.find(".//metaItem[@about='name']")
    if name_section is not None:
        names['Main Name'] = extract_name(name_section, 'mainName')
        names['Forenames'] = extract_name(name_section, 'foreNames')
        names['ShortForm'] = extract_name(name_section, 'ShortForm')
    alternative_name_sections = root.findall(".//metaItem[@about='alternative_name']")
    for alt_name_section in alternative_name_sections:
        main_name = extract_name(alt_name_section, 'mainName')
        forenames = extract_name(alt_name_section, 'foreNames')
        if main_name != 'N/A' and main_name not in names['Alternative Names'] and main_name != names['Main Name']:
            names['Alternative Names'].append(main_name)
        if forenames != 'N/A' and forenames not in names['Alternative Names'] and forenames != names['Forenames']:
            names['Alternative Names'].append(forenames)
    names['Alternative Names'] = ', '.join(names['Alternative Names'])

    return names['Alternative Names']

In [3]:
def process(xmlfile):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    data_dict = {}
    def extract_text(element):
        return element.text if element is not None else "N/A"
    creator_group = root.find(".//creatorGroup")
    if creator_group is not None:
        data_dict["Author_Forenames"] = extract_text(creator_group.find("creator/forenames"))
        data_dict["Author_Surname"] = extract_text(creator_group.find("creator/surname"))
    else:
        data_dict["Author_Forenames"] = "N/A"
        data_dict["Author_Surname"] = "N/A"
    parents = root.findall(".//metaItem[@about='parents']/metaItem[@about='parent']")
    parent_data = []
    for parent in parents:
        parent_info = {}
        parent_info["Gender"] = extract_text(parent.find(".//metaDescribes[@predicate='gender']"))
        parent_info["MainName"] = extract_text(parent.find(".//metaDescribes[@predicate='mainName']"))
        parent_info["ForeNames"] = extract_text(parent.find(".//metaDescribes[@predicate='foreNames']"))
        parent_info["Birth"] = extract_text(parent.find(".//metaItem[@about='birth']/metaDescribes[@predicate='date']"))
        parent_info["Death"] = extract_text(parent.find(".//metaItem[@about='death']/metaDescribes[@predicate='date']"))
        parent_info["Occupation"] = extract_text(parent.find(".//metaDescribes[@predicate='occupation']"))
        parent_data.append(parent_info)
    profile = root.find(".//metaItem[@about='profile']")
    if profile is not None:
        data_dict["Gender"] = extract_text(profile.find(".//metaDescribes[@predicate='gender']"))
        data_dict["MainName"] = extract_text(profile.find(".//metaDescribes[@predicate='mainName']"))
        data_dict["foreNames"] = extract_text(profile.find(".//metaDescribes[@predicate='foreNames']"))
        data_dict["ShortForm"] = extract_text(profile.find(".//metaDescribes[@predicate='ShortForm']"))
        data_dict["Birth"] = extract_text(profile.find(".//metaItem[@about='birth']/metaDescribes[@predicate='date']"))
        data_dict["Birth Place - Street"] = extract_text(profile.find(".//metaItem[@about='birth']/metaItem/metaDescribes[@predicate='street']"))
        data_dict["Birth Place - Town"] = extract_text(profile.find(".//metaItem[@about='birth']/metaItem/metaDescribes[@predicate='town']"))
        data_dict["Birth Place - City"] = extract_text(profile.find(".//metaItem[@about='birth']/metaItem/metaDescribes[@predicate='city']"))
        data_dict["Birth Place - County"] = extract_text(profile.find(".//metaItem[@about='birth']/metaItem/metaDescribes[@predicate='county']"))
        data_dict["Birth Place - Country"] = extract_text(profile.find(".//metaItem[@about='birth']/metaItem/metaDescribes[@predicate='country']"))
        data_dict["Death"] = extract_text(profile.find(".//metaItem[@about='death']/metaDescribes[@predicate='date']"))
        data_dict["Death Place - Street"] = extract_text(profile.find(".//metaItem[@about='death']/metaItem/metaDescribes[@predicate='street']"))
        data_dict["Death Place - Town"] = extract_text(profile.find(".//metaItem[@about='death']/metaItem/metaDescribes[@predicate='town']"))
        data_dict["Death Place - City"] = extract_text(profile.find(".//metaItem[@about='death']/metaItem/metaDescribes[@predicate='city']"))
        data_dict["Death Place - County"] = extract_text(profile.find(".//metaItem[@about='death']/metaItem/metaDescribes[@predicate='county']"))
        data_dict["Death Place - Country"] = extract_text(profile.find(".//metaItem[@about='death']/metaItem/metaDescribes[@predicate='country']"))

        occupations = extract_person_occupations(xmlfile)
        data_dict["Occupation"] = ', '.join(occupations) if occupations else 'N/A'

        alternative_names = extract_names(xmlfile, print_names=True)
        data_dict["Alternative Names"] = alternative_names

    else:
        data_dict["Gender"] = "N/A"
        data_dict["MainName"] = "N/A"
        data_dict["foreNames"] = "N/A"
        data_dict["ShortForm"] = "N/A"
        data_dict["Birth"] = "N/A"
        data_dict["Birth Place - Street"] = "N/A"
        data_dict["Birth Place - City"] = "N/A"
        data_dict["Birth Place - Country"] = "N/A"
        data_dict["Birth Place - Town"] = "N/A"
        data_dict["Birth Place - County"] = "N/A"
        data_dict["Death"] = "N/A"
        data_dict["Death Place - Street"] = "N/A"
        data_dict["Death Place - Town"] = "N/A"
        data_dict["Death Place - County"] = "N/A"
        data_dict["Death Place - City"] = "N/A"
        data_dict["Death Place - Country"] = "N/A"
        data_dict["Occupation"] = "N/A"
        data_dict["Alternative Names"] = "N/A"

    # Add parent data to the dictionary
    for i, parent_info in enumerate(parent_data, start=1):
        for key, value in parent_info.items():
            if i == 1:
                data_dict[f"Father_{key}"] = value
            elif i == 2:
                data_dict[f"Mother_{key}"] = value

    # Initialize data dictionary with empty school and college columns
    school_columns = []
    college_columns = []

    # Extract education data
    education_data = root.findall(".//metaItem[@about='education']")
    for idx, education_item in enumerate(education_data):
        school = education_item.find(".//metaItem[@about='school']")
        if school is not None:
            school_name = extract_text(school.find(".//metaDescribes[@predicate='institution']"))
            school_city = extract_text(school.find(".//metaDescribes[@predicate='city']"))
            school_country = extract_text(school.find(".//metaDescribes[@predicate='country']"))
            data_dict[f"School_{idx+1}"] = school_name
            data_dict[f"School_{idx+1}_City"] = school_city
            data_dict[f"School_{idx+1}_Country"] = school_country

        college = education_item.find(".//metaItem[@about='college']")
        if college is not None:
            college_name = extract_text(college.find(".//metaDescribes[@predicate='institution']"))
            college_city = extract_text(college.find(".//metaDescribes[@predicate='town']"))
            college_country = extract_text(college.find(".//metaDescribes[@predicate='country']"))
            data_dict[f"College_{idx+1}"] = college_name
            data_dict[f"College_{idx+1}_City"] = college_city
            data_dict[f"College_{idx+1}_Country"] = college_country

    # Extract residence data
    residence = root.find(".//metaItem[@about='residence']")
    if residence is not None:
        data_dict["Residence_Date"] = extract_text(residence.find(".//metaDescribes[@predicate='date']"))
        data_dict["Residence_Town"] = extract_text(residence.find(".//metaItem/metaDescribes[@predicate='town']"))
        data_dict["Residence_Street"] = extract_text(residence.find(".//metaItem/metaDescribes[@predicate='street']"))
        data_dict["Residence_City"] = extract_text(residence.find(".//metaItem/metaDescribes[@predicate='city']"))
        data_dict["Residence_County"] = extract_text(residence.find(".//metaItem/metaDescribes[@predicate='county']"))
        data_dict["Residence_Country"] = extract_text(residence.find(".//metaItem/metaDescribes[@predicate='country']"))
    else:
        data_dict["Residence_Date"] = "N/A"
        data_dict["Residence_Town"] = "N/A"
        data_dict["Residence_Street"] = 'N/A'
        data_dict["Residence_City"] = 'N/A'
        data_dict["Residence_County"] = "N/A"
        data_dict["Residence_Country"] = "N/A"

    # Initialize an empty list to store taxonomy terms
    taxonomy_terms_list = []

    # Extract taxonomy data
    taxonomy_data = root.findall(".//metaItem[@about='taxonomyTerm']")
    for taxonomy_term in taxonomy_data:
        taxonomy = extract_text(taxonomy_term.find(".//metaDescribes[@predicate='taxonomy']"))
        code = extract_text(taxonomy_term.find(".//metaDescribes[@predicate='code']"))
        # Combine taxonomy and code into a single string
        combined_taxonomy = f"{taxonomy} ({code})"
        taxonomy_terms_list.append(combined_taxonomy)

    # Join the taxonomy terms into a single string separated by commas
    data_dict["Taxonomy_Terms"] = ', '.join(taxonomy_terms_list)

    # Initialize an empty list to store institution names
    institution_names = []

    # Extract institution data
    institution_data = root.findall(".//place[@institution]")
    for institution_item in institution_data:
        institution_name = institution_item.get("institution")
        if institution_name and institution_name not in institution_names:
            institution_names.append(institution_name)

    # Join the institution names into a single string separated by commas
    data_dict["Institutions"] = ', '.join(institution_names)

    # Flag to indicate whether data was successfully extracted using format 1
    format_1_success = False

    # Extract wealth at death data using format 1
    wealth_data = root.find(".//section[@role='wealthAtDeath']")
    if wealth_data is not None:
        wealth_amount = extract_text(wealth_data.find(".//span[@role='money']"))
        wealth_probate_date = extract_text(wealth_data.find(".//p"))
        wealth_abbrev = extract_text(wealth_data.find(".//abbrev"))

        # If data is successfully extracted using format 1, set the flag
        if wealth_amount != "N/A" and wealth_probate_date != "N/A":
            format_1_success = True

    # If data is not successfully extracted using format 1, try format 2
    if not format_1_success:
        # Extract wealth at death data using format 2
        wealth_data = root.find(".//section[@role='wealthAtDeath']")
        if wealth_data is not None:
            # Check if the new format is present
            div1 = wealth_data.find(".//div1")
            if div1 is not None:
                # Extract wealth amount from the new format
                wealth_p_tag = div1.find("./p")
                if wealth_p_tag is not None and wealth_p_tag.text is not None:
                    wealth_text = wealth_p_tag.text
                    wealth_amount_match = re.search(r'£([\d,]+)', wealth_text)
                    if wealth_amount_match:
                        wealth_amount = wealth_amount_match.group(1).replace(',', '')
                    else:
                        wealth_amount = "N/A"
                else:
                    wealth_amount = "N/A"
            else:
                # Extract wealth amount from the original format
                wealth_amount = extract_text(wealth_data.find(".//span[@role='money']"))
            # Extract other relevant data (if applicable)
            wealth_probate_date = extract_text(wealth_data.find(".//p"))
            wealth_abbrev = extract_text(wealth_data.find(".//abbrev"))

        else:
            wealth_amount = "N/A"
            wealth_probate_date = "N/A"
            wealth_abbrev = "N/A"

    # Add wealth at death data to the dictionary
    data_dict["Wealth_Amount"] = wealth_amount
    data_dict["Wealth_Probate_Date"] = wealth_probate_date
    data_dict["Wealth_Abbreviation"] = wealth_abbrev


    # Extract spouse data
    spouses = root.findall(".//metaItem[@about='spouses']/metaItem[@about='spouse']")
    spouse_data = []
    for spouse in spouses:
        spouse_info = {}
        spouse_info["Gender"] = extract_text(spouse.find(".//metaDescribes[@predicate='gender']"))
        spouse_info["MainName"] = extract_text(spouse.find(".//metaDescribes[@predicate='mainName']"))
        spouse_info["ForeNames"] = extract_text(spouse.find(".//metaDescribes[@predicate='foreNames']"))
        spouse_info["Birth"] = extract_text(spouse.find(".//metaItem[@about='birth']/metaDescribes[@predicate='date']"))
        spouse_info["Death"] = extract_text(spouse.find(".//metaItem[@about='death']/metaDescribes[@predicate='date']"))
        spouse_info["Occupation"] = extract_text(spouse.find(".//metaDescribes[@predicate='occupation']"))
        spouse_info["Relationship_Date"] = extract_text(spouse.find(".//metaItem[@about='relationship_dates']/metaDescribes[@predicate='date']"))
        spouse_data.append(spouse_info)

    # Add spouse data to the dictionary
    for i, spouse_info in enumerate(spouse_data, start=1):
        for key, value in spouse_info.items():
            data_dict[f"Spouse_{i}_{key}"] = value

    # Initialize spouse columns in DataFrame
    spouse_columns = []
    if spouse_data:
        spouse_columns = [f"Spouse_{i}_{key}" for key in spouse_data[0].keys() for i in range(1, len(spouse_data)+1)]

    # Update spouse columns in DataFrame
    for col in spouse_columns:
        if col not in data_dict:
            data_dict[col] = "N/A"

    return data_dict

In [4]:
def main(directory):
    log_file_path = os.path.join(directory, 'log.txt')
    all_data = []
    if not os.path.exists(log_file_path):
        open(log_file_path, 'w').close()
    with open(log_file_path, 'r') as file:
        processed_files = set(file.read().splitlines())
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.xml') and filename not in processed_files:
            file_path = os.path.join(directory, filename)
            file_data = process(file_path)
            all_data.append(file_data)
            with open(log_file_path, 'a') as log_file:
                log_file.write(filename + '\n')
    df = pd.DataFrame(all_data)
    df.to_excel("dataset_new.xlsx", index=False)
if __name__ == "__main__":
    directory = r'C:\Users\Muneeb Nabeel\Desktop\FINAL desertation code files\combined_xml'
    main(directory)

100%|███████████████████████████████████████████████████████████████████████████| 39518/39518 [01:40<00:00, 392.92it/s]


In [6]:
df = pd.read_excel('dataset_new.xlsx')
# Display basic information about the dataset
print("Dataset Info:")
print(df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39517 entries, 0 to 39516
Columns: 136 entries, Author_Forenames to Spouse_12_Relationship_Date
dtypes: float64(30), object(106)
memory usage: 41.0+ MB
None


In [21]:
pip install pandas sqlalchemy pyodbc

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server']


In [11]:
import pyodbc

try:
    conn = pyodbc.connect(
        r'DRIVER={ODBC Driver 17 for SQL Server};'
        r'SERVER=MUNEEB_LENOVO;'  # Replace with your server
        r'DATABASE=Desertation;'  # Replace with your database
        r'Trusted_Connection=yes;'
    )
    print("Connection successful!")
except Exception as e:
    print("Error occurred:", e)

Connection successful!


In [13]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('mssql+pyodbc://@MUNEEB_LENOVO/Desertation?driver=ODBC Driver 17 for SQL Server;trusted_connection=yes')
# df = pd.read_sql_query("SELECT * FROM your_table", engine)
# print(df)

In [15]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine

def check_drivers():
    print("Available ODBC drivers:")
    for driver in pyodbc.drivers():
        print(driver)
    print("\n")

def test_pyodbc_connection(server, database):
    try:
        connection_string = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={server};"
            f"DATABASE={database};"
            "Trusted_Connection=yes;"
        )
        conn = pyodbc.connect(connection_string)
        print("Direct pyodbc connection successful.")
        conn.close()
    except Exception as e:
        print("Failed to connect using pyodbc:", str(e))
    print("\n")

def test_pyodbc_query(server, database):
    try:
        connection_string = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={server};"
            f"DATABASE={database};"
            "Trusted_Connection=yes;"
        )
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        cursor.execute("SELECT 1")  # Simple query
        print("Query executed successfully:", cursor.fetchone())
        conn.close()
    except Exception as e:
        print("Failed to execute query using pyodbc:", str(e))
    print("\n")

def test_sqlalchemy_connection(connection_string):
    try:
        engine = create_engine(connection_string, echo=True)
        with engine.connect() as conn:
            result = conn.execute("SELECT 1")
            print("SQLAlchemy connection successful. Query result:", result.fetchone())
    except Exception as e:
        print("Failed to connect or query using SQLAlchemy:", str(e))
    print("\n")

def main():
    server = 'MUNEEB_LENOVO'
    database = 'Desertation'

    # Check available drivers
    check_drivers()

    # Test connection using pyodbc
    test_pyodbc_connection(server, database)

    # Test query execution using pyodbc
    test_pyodbc_query(server, database)

    # Build connection string for SQLAlchemy
    connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes;driver=ODBC Driver 17 for SQL Server'

    # Test connection using SQLAlchemy
    test_sqlalchemy_connection(connection_string)

if __name__ == "__main__":
    main()


Available ODBC drivers:
SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access Text Driver (*.txt, *.csv)
SQL Server Native Client RDA 11.0
ODBC Driver 17 for SQL Server
ODBC Driver 18 for SQL Server


Direct pyodbc connection successful.


Query executed successfully: (1, )


Failed to connect or query using SQLAlchemy: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/rvf5)




C:\Users\Muneeb Nabeel\AppData\Local\Temp\ipykernel_10048\1999813649.py:45: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  engine = create_engine(connection_string, echo=True)


In [16]:
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)
print("Connection successful!")
conn.close()

Connection successful!


In [17]:
from sqlalchemy import create_engine
engine = create_engine(f'mssql+pyodbc://{server}/{database}?trusted_connection=yes;driver=ODBC Driver 17 for SQL Server', echo=True)
with engine.connect() as conn:
    result = conn.execute("SELECT 1")
    print("Query result:", result.fetchone())

C:\Users\Muneeb Nabeel\AppData\Local\Temp\ipykernel_10048\2984008719.py:2: SAWarning: No driver name specified; this is expected by PyODBC when using DSN-less connections
  engine = create_engine(f'mssql+pyodbc://{server}/{database}?trusted_connection=yes;driver=ODBC Driver 17 for SQL Server', echo=True)


InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/rvf5)

In [18]:
import pyodbc
print("Available ODBC Drivers:", pyodbc.drivers())

Available ODBC Drivers: ['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server', 'ODBC Driver 18 for SQL Server']


In [19]:
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=MUNEEB_LENOVO;"
    "DATABASE=Desertation;"
    "Trusted_Connection=yes;"
)
try:
    conn = pyodbc.connect(connection_string)
    print("Connection successful!")
    conn.close()
except Exception as e:
    print("Failed to connect using pyodbc:", e)


Connection successful!


In [32]:
from sqlalchemy import create_engine, engine
from sqlalchemy.engine.url import URL

connection_url = URL.create(
    "mssql+pyodbc",
    username="",
    password="",
    host=server,
    database=database,
    query={
        "trusted_connection": "yes",
        "driver": "ODBC Driver 17 for SQL Server"
    }
)

engine = create_engine(connection_url, echo=True)

try:
    with engine.connect() as conn:
        result = conn.execute("SELECT 1")
        print("Query result:", result.fetchone())
except Exception as e:
    print("SQLAlchemy connection error:", e)

2024-04-14 17:17:04,080 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-04-14 17:17:04,080 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-14 17:17:04,080 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-04-14 17:17:04,090 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ()
2024-04-14 17:17:04,100 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-04-14 17:17:04,100 INFO sqlalchemy.engine.Engine [generated in 0.00098s] ()
2024-04-14 17:17:04,100 INFO sqlalchemy.engine.Engine SELECT 1
2024-04-14 17:17:04,100 INFO sqlalchemy.engine.Engine [raw sql] ()
Query result: (1,)


In [34]:
import pandas as pd
from sqlalchemy import create_engine, engine
from sqlalchemy.engine.url import URL

# Load the dataset from Excel
df = pd.read_excel('dataset_new.xlsx')

# Database connection parameters
server = 'MUNEEB_LENOVO'
database = 'Desertation'
driver = 'ODBC Driver 17 for SQL Server'

# Using URL object to ensure proper formatting
connection_url = URL.create(
    "mssql+pyodbc",
    username="",  # Not needed for Windows Authentication
    password="",  # Not needed for Windows Authentication
    host=server,
    database=database,
    query={
        "trusted_connection": "yes",
        "driver": driver
    }
)

# Create the SQLAlchemy engine with verbose SQL output
engine = create_engine(connection_url, echo=True)

# Write the DataFrame to SQL Server in the specified table 'MyRawDataTable'
try:
    df.to_sql('MyRawDataTable', con=engine, index=False, if_exists='replace')
    print("Data successfully written to SQL Server")
except Exception as e:
    print("Failed to write data to SQL Server:", e)


2024-04-14 17:22:34,113 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-04-14 17:22:34,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-14 17:22:34,117 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-04-14 17:22:34,117 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ()
2024-04-14 17:22:34,134 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-04-14 17:22:34,134 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ()
2024-04-14 17:22:34,134 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-14 17:22:34,337 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-04-14 17:22:34,340 INFO sqlalchemy.engi

2024-04-14 17:22:38,174 INFO sqlalchemy.engine.Engine [generated in 2.81563s] (('Veronica', 'Kelly', 'f', 'Brayton', 'Lily', 'Brayton, Elizabeth [Lily]', '1876-06-23', 'White House, Stony Lane', 'Hindley', None, 'Lancashire', 'England', '1953-04-30', 'Redcliffe, East Cliffe Road', 'Dawlish', None, 'Devon', 'England', 'actor and theatrical producer', 'Elizabeth, Asche, Watson, Chalmers, Douglas', 'm', 'Brayton', 'John Grindall', '1842', '1892', 'medical practitioner', 'f', 'Alexander', 'Margaret', None, None, None, '1953', 'Dawlish', 'Redcliffe, East Cliffe Road', None, 'Devon', 'England', 'OccupationsAndRealmsOfRenown (732)', None, '£49,336 15', 'Wealth at Death', None, 'm', 'Asche', 'Thomas Stange Heiss Oscar', '1871', '1936', None, '1898-06-22', 'm', 'Watson', 'Douglas Chalmers', '1870', '1946', 'physician', '1938-06-15', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

2024-04-14 17:27:05,534 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) ORDER BY [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME]
2024-04-14 17:27:05,534 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('dbo', 'BASE TABLE')
2024-04-14 17:27:05,558 INFO sqlalchemy.engine.Engine COMMIT
Data successfully written to SQL Server


# 2. Cleaning and Structuring Raw Data for Analysis

In [ ]:
# Data Transformation: Category aggregation: Group similar categories together to simplify the analysis.

## Spouse

In [7]:
# Define a list to store spouse-related columns
spouse_columns = []

# Define a list to store all columns
all_columns = []

# Iterate through numbers from 1 to 12
for i in range(1, 13):
    # Construct column names using string formatting
    gender_column = f"Spouse_{i}_Gender"
    main_name_column = f"Spouse_{i}_MainName"
    forenames_column = f"Spouse_{i}_ForeNames"
    birth_column = f"Spouse_{i}_Birth"
    death_column = f"Spouse_{i}_Death"
    occupation_column = f"Spouse_{i}_Occupation"
    relationship_date_column = f"Spouse_{i}_Relationship_Date"
    
    # Add constructed columns to the list
    spouse_columns.extend([gender_column, main_name_column, forenames_column, 
                           birth_column, death_column, occupation_column, 
                           relationship_date_column])

# Iterate through all spouse columns and get all unique columns
for column in spouse_columns:
    all_columns.append(column)

# Get unique columns and sort them
all_columns = sorted(set(all_columns))

# Define a list to store only spouse name related columns
spouse_name_columns = []

# Define a list to store columns not related to names
non_name_related_columns = []

# Filter out spouse name related columns
for column in all_columns:
    if "MainName" in column or "ForeNames" in column:
        spouse_name_columns.append(column)
    else:
        non_name_related_columns.append(column)

# Print the spouse name related columns
print("Spouse Name Related Columns:")
for column in spouse_name_columns:
    print(column)

# Print the columns not related to the names
print("\nColumns not related to names:")
for column in non_name_related_columns:
    print(column)

Spouse Name Related Columns:
Spouse_10_ForeNames
Spouse_10_MainName
Spouse_11_ForeNames
Spouse_11_MainName
Spouse_12_ForeNames
Spouse_12_MainName
Spouse_1_ForeNames
Spouse_1_MainName
Spouse_2_ForeNames
Spouse_2_MainName
Spouse_3_ForeNames
Spouse_3_MainName
Spouse_4_ForeNames
Spouse_4_MainName
Spouse_5_ForeNames
Spouse_5_MainName
Spouse_6_ForeNames
Spouse_6_MainName
Spouse_7_ForeNames
Spouse_7_MainName
Spouse_8_ForeNames
Spouse_8_MainName
Spouse_9_ForeNames
Spouse_9_MainName

Columns not related to names:
Spouse_10_Birth
Spouse_10_Death
Spouse_10_Gender
Spouse_10_Occupation
Spouse_10_Relationship_Date
Spouse_11_Birth
Spouse_11_Death
Spouse_11_Gender
Spouse_11_Occupation
Spouse_11_Relationship_Date
Spouse_12_Birth
Spouse_12_Death
Spouse_12_Gender
Spouse_12_Occupation
Spouse_12_Relationship_Date
Spouse_1_Birth
Spouse_1_Death
Spouse_1_Gender
Spouse_1_Occupation
Spouse_1_Relationship_Date
Spouse_2_Birth
Spouse_2_Death
Spouse_2_Gender
Spouse_2_Occupation
Spouse_2_Relationship_Date
Spouse_3_B

In [8]:
# Define the list of columns not related to names
columns_to_drop = [
    f"Spouse_{i}_Birth" for i in range(1, 13)
] + [
    f"Spouse_{i}_Death" for i in range(1, 13)
] + [
    f"Spouse_{i}_Gender" for i in range(1, 13)
] + [
    f"Spouse_{i}_Occupation" for i in range(1, 13)
] + [
    f"Spouse_{i}_Relationship_Date" for i in range(1, 13)
]

# Drop the columns from the DataFrame
df = df.drop(columns=columns_to_drop)

In [9]:
# Create empty lists to store spouse names and count
spouse_names = []
no_of_spouses = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Initialize variables for spouse names and count for current row
    current_spouse_names = []
    current_no_of_spouses = 0
    
    # Iterate through spouse-related columns
    for i in range(1, 13):
        main_name_col = f"Spouse_{i}_MainName"
        forenames_col = f"Spouse_{i}_ForeNames"
        
        # Check if both MainName and ForeNames are available
        if pd.notna(row[main_name_col]) and pd.notna(row[forenames_col]):
            # Concatenate MainName and ForeNames with a space in between
            current_spouse_names.append(f"{row[forenames_col]} {row[main_name_col]}")
        # Check if only MainName is available
        elif pd.notna(row[main_name_col]):
            current_spouse_names.append(row[main_name_col])
        # Check if only ForeNames is available
        elif pd.notna(row[forenames_col]):
            current_spouse_names.append(row[forenames_col])
        
        # Increment spouse count if either MainName or ForeNames is available
        if pd.notna(row[main_name_col]) or pd.notna(row[forenames_col]):
            current_no_of_spouses += 1
    
    # Append the concatenated spouse names and count for current row
    spouse_names.append(", ".join(current_spouse_names))
    no_of_spouses.append(current_no_of_spouses)

# Create a temporary DataFrame to hold the computed values
temp_df = pd.DataFrame({
    'Spouse_Names': spouse_names,
    'No_of_Spouse': no_of_spouses
}, index=df.index)

# Concatenate the original DataFrame with the temporary DataFrame
df = pd.concat([df, temp_df], axis=1)

# Print the DataFrame to verify the changes
print(df)

      Author_Forenames Author_Surname Gender     MainName        foreNames  \
0             Veronica          Kelly      f      Brayton             Lily   
1            Maggie B.           Gale      f      Collier        Constance   
2                  Rob          Steen      m        Close           Dennis   
3             Rosemary          Scott      m       Barham   Richard Harris   
4                R. K.           Webb      m       Barham    Thomas Foster   
...                ...            ...    ...          ...              ...   
39512          Todd M.       Thompson      m         Kerr      David Allan   
39513             Gill      Coleridge      m      Kington  Miles Beresford   
39514            Tessa        Murdoch      m  Fourdrinier             Paul   
39515        Ronald T.         Raines      m      Knowles   Jeremy Randall   
39516              NaN            NaN      m        Laing     John Maurice   

                       ShortForm       Birth         Birth Plac

In [11]:
# Iterate through spouse-related columns
for i in range(1, 13):
    main_name_col = f"Spouse_{i}_MainName"
    forenames_col = f"Spouse_{i}_ForeNames"
    
    # Drop the old spouse-related columns
    df.drop(columns=[main_name_col, forenames_col], inplace=True)

# Print the DataFrame to verify the changes
print(df)

KeyError: "['Spouse_1_MainName', 'Spouse_1_ForeNames'] not found in axis"

## Schools

In [12]:
# Create empty lists to store school names and counts
schools = []
no_of_schools = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Initialize variables for school names and count for current row
    current_schools = []
    current_no_of_schools = 0
    
    # Iterate through school-related columns
    for i in range(1, 3):
        school_col = f"School_{i}"
        
        # Check if school name is available
        if pd.notna(row[school_col]):
            # Append school name to the list
            current_schools.append(row[school_col])
            current_no_of_schools += 1
    
    # Append the concatenated school names and count for current row
    schools.append(", ".join(current_schools))
    no_of_schools.append(current_no_of_schools)

# Create a temporary DataFrame to hold the computed values
temp_df_school = pd.DataFrame({
    'Schools': schools,
    'No_of_Schools': no_of_schools
}, index=df.index)

# Concatenate the original DataFrame with the temporary DataFrame
df = pd.concat([df, temp_df_school], axis=1)

# # Drop unnecessary city and country columns
# columns_to_drop = ['School_1_City', 'School_1_Country', 'School_2_City', 'School_2_Country','College_1_City','College_1_Country']
# df.drop(columns=columns_to_drop, inplace=True)

# Print the DataFrame to verify the changes
print(df)


      Author_Forenames Author_Surname Gender     MainName        foreNames  \
0             Veronica          Kelly      f      Brayton             Lily   
1            Maggie B.           Gale      f      Collier        Constance   
2                  Rob          Steen      m        Close           Dennis   
3             Rosemary          Scott      m       Barham   Richard Harris   
4                R. K.           Webb      m       Barham    Thomas Foster   
...                ...            ...    ...          ...              ...   
39512          Todd M.       Thompson      m         Kerr      David Allan   
39513             Gill      Coleridge      m      Kington  Miles Beresford   
39514            Tessa        Murdoch      m  Fourdrinier             Paul   
39515        Ronald T.         Raines      m      Knowles   Jeremy Randall   
39516              NaN            NaN      m        Laing     John Maurice   

                       ShortForm       Birth         Birth Plac

In [13]:
# Define the columns to drop
columns_to_drop = ['School_1', 'School_2']
# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)


## Author

In [14]:
# Define the columns to drop
columns_to_drop = ['Author_Forenames', 'Author_Surname']
# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)


## Parents

In [15]:
# Define the columns to drop
columns_to_drop = ['Father_Gender', 'Mother_Gender']
# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

## Columns with Missing Values

In [16]:
# Display missing values for all columns
print("\nMissing Values:")
missing_values = df.isnull().sum()
for column, missing_count in missing_values.items():
    print(f"{column}: {missing_count}")


Missing Values:
Gender: 2032
MainName: 1294
foreNames: 3092
ShortForm: 39472
Birth: 11081
Birth Place - Street: 31193
Birth Place - Town: 17498
Birth Place - City: 39186
Birth Place - County: 15136
Birth Place - Country: 12915
Death: 5328
Death Place - Street: 25112
Death Place - Town: 15291
Death Place - City: 39227
Death Place - County: 12768
Death Place - Country: 10317
Occupation: 5801
Alternative Names: 22932
Father_MainName: 7980
Father_ForeNames: 9679
Father_Birth: 38972
Father_Death: 38932
Father_Occupation: 13632
Mother_MainName: 13390
Mother_ForeNames: 14450
Mother_Birth: 39020
Mother_Death: 38985
Mother_Occupation: 37138
Residence_Date: 39068
Residence_Town: 15432
Residence_Street: 28042
Residence_City: 39219
Residence_County: 9205
Residence_Country: 5804
Taxonomy_Terms: 1817
Institutions: 10647
Wealth_Amount: 25612
Wealth_Probate_Date: 23995
Wealth_Abbreviation: 39506
School_1_City: 39281
School_1_Country: 25746
College_1: 24585
College_1_City: 26612
College_1_Country: 248

In [17]:
# Define the columns to drop
columns_to_drop = ['ShortForm','Birth Place - Street','Death Place - Street','Father_Birth','Father_Death','Mother_Birth','Mother_Death']
# Drop the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

## Date Conversion

In [18]:
# A more robust function to extract the year from various date formats
def extract_year(date_str):
    if pd.isnull(date_str):
        return None
    try:
        # First attempt to parse the date using to_datetime, which will correctly
        # interpret different date formats like "YYYY-MM-DD" or "MM/DD/YYYY".
        return pd.to_datetime(date_str, errors='coerce').year
    except ValueError:
        pass  # Ignore the ValueError and attempt further processing
    
    # If to_datetime failed, manually parse the string
    try:
        parts = date_str.replace('-', '/').split('/')
        # If the length of the first part is 4, it's likely the year
        if len(parts[0]) == 4:
            return int(parts[0])
        # If the length of the last part is 4, it's likely the year
        elif len(parts[-1]) == 4:
            return int(parts[-1])
        else:
            # If year is not found, return None
            return None
    except Exception as e:
        # In case of any other unexpected format or error
        return None


# Assuming 'df' is your DataFrame after loading the CSV:
df['Birth_Year'] = df['Birth'].apply(extract_year)
df['Death_Year'] = df['Death'].apply(extract_year)

# Calculate the age by subtracting the Birth year from the Death year
df['Age'] = df['Death_Year'] - df['Birth_Year']

# Display the DataFrame to verify the results
df[['Birth', 'Death', 'Birth_Year', 'Death_Year', 'Age']]

,Birth,Death,Birth_Year,Death_Year,Age
0,1876-06-23,1953-04-30,1876.0,1953.0,77.0
1,1878-01-22,1955-04-25,1878.0,1955.0,77.0
2,1931-02-24,2015-09-13,1931.0,2015.0,84.0
3,1788-12-06,1845-06-17,1788.0,1845.0,57.0
4,1794-09-10,1869-03-03,1794.0,1869.0,75.0
...,...,...,...,...,...
39512,1945-05-16,2008-04-14,1945.0,2008.0,63.0
39513,1941-05-13,2008-01-30,1941.0,2008.0,67.0
39514,1698,NaN,1698.0,NaN,NaN
39515,1935-04-28,2008-04-03,1935.0,2008.0,73.0


In [19]:
# Combine the mainname and forename columns
df['Name'] = df['foreNames'] + ' ' + df['MainName']

# 3. Saving the processed data to a new file

In [20]:
df.to_csv('Final_Dataset.csv', index=False)